**Exercise for Unit 6 - NLP**

**1. (70 points) Implement a Hidden Markov model in Python, using the training datasets given on the Assignment for Unit 6.**

In [36]:
from collections import defaultdict, Counter
import math

In [37]:
# Step 1: Parse dataset
data = [
    "The_DET cat_NOUN sleeps_VERB",
    "A_DET dog_NOUN barks_VERB",
    "The_DET dog_NOUN sleeps_VERB",
    "My_DET dog_NOUN runs_VERB fast_ADV",
    "A_DET cat_NOUN meows_VERB loudly_ADV",
    "Your_DET cat_NOUN runs_VERB",
    "The_DET bird_NOUN sings_VERB sweetly_ADV",
    "A_DET bird_NOUN chirps_VERB"
]

In [38]:
tag_bigrams = defaultdict(Counter)
emissions = defaultdict(Counter)
start_tags = Counter()
tags = set()

In [39]:
# Step 2: Extract counts from dataset
for sentence in data:
    words_tags = sentence.split()
    prev_tag = None
    for i, wt in enumerate(words_tags):
        word, tag = wt.rsplit("_", 1)
        emissions[tag][word.lower()] += 1  # convert to lowercase
        tags.add(tag)
        if i == 0:
            start_tags[tag] += 1
        if prev_tag is not None:
            tag_bigrams[prev_tag][tag] += 1
        prev_tag = tag

In [40]:
# Step 3: Convert counts to probabilities
def normalize(counter):
    total = sum(counter.values())
    return {key: val / total for key, val in counter.items()}

initial_probs = normalize(start_tags)
transition_probs = {tag: normalize(counter) for tag, counter in tag_bigrams.items()}
emission_probs = {tag: normalize(counter) for tag, counter in emissions.items()}

In [41]:
# Step 4: Viterbi algorithm
def viterbi(observation, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    for y in states:
        V[0][y] = math.log(start_p.get(y, 1e-10)) + math.log(emit_p.get(y, {}).get(observation[0], 1e-10))
        path[y] = [y]

    for t in range(1, len(observation)):
        V.append({})
        new_path = {}
        for y in states:
            (prob, state) = max(
                (V[t-1][y0] + math.log(trans_p.get(y0, {}).get(y, 1e-10)) +
                 math.log(emit_p.get(y, {}).get(observation[t], 1e-10)), y0)
                for y0 in states
            )
            V[t][y] = prob
            new_path[y] = path[state] + [y]
        path = new_path

    n = len(observation) - 1
    (prob, state) = max((V[n][y], y) for y in states)
    return path[state]

**User Input**

In [42]:
# Step 5: User input
user_input = input("Enter a sentence: ").strip().lower()
user_words = user_input.split()

In [43]:
# Step 6: Predict tags
predicted_tags = viterbi(user_words, tags, initial_probs, transition_probs, emission_probs)
print (predicted_tags)

['DET', 'NOUN', 'VERB']


In [44]:
# Step 7: Output
print("\nPredicted POS tags:")
for word, tag in zip(user_words, predicted_tags):
    print(f"{word} -> {tag}")


Predicted POS tags:
the -> DET
dog -> NOUN
barks -> VERB


**2.	(30 points) Implement Viterbi algorithm to determine the best PoS sequence on the following test sentences:**

**a.	The can meows**

**b.	My dog barks loudly**


In [45]:
# Test sentences
test_sentences = {
    "a": "The can meows",
    "b": "My dog barks loudly"
}

In [46]:
# Run Viterbi on each
for label, sentence in test_sentences.items():
    words = sentence.lower().split()
    tags_seq = viterbi(words, tags, initial_probs, transition_probs, emission_probs)
    print(f"\nSentence {label}: {sentence}")
    for word, tag in zip(words, tags_seq):
        print(f"{word} -> {tag}")


Sentence a: The can meows
the -> DET
can -> NOUN
meows -> VERB

Sentence b: My dog barks loudly
my -> DET
dog -> NOUN
barks -> VERB
loudly -> ADV
